

# MVP de Engenharia e Análise de Dados: Impacto das Redes Sociais na Saúde Mental

**Aluno:** Eric Koji Nakirimoto

## 1. Visão Geral do Projeto
Este projeto tem como objetivo construir um pipeline de dados completo (End-to-End) na nuvem utilizando a plataforma **Databricks free**. O foco central é analisar a correlação entre hábitos digitais — especificamente o uso de redes sociais e tempo de tela — e indicadores de saúde mental e bem-estar, como níveis de estresse, qualidade do sono e felicidade.

A análise busca responder se o estilo de vida digital moderno está impactando negativamente a saúde emocional dos usuários, fornecendo insights baseados em dados para uma discussão mais fundamentada sobre o tema.

## 2. Perguntas de Negócio (Insights)
Para guiar a modelagem e a análise dos dados, foram definidas as seguintes questões norteadoras:
1.  **Redes Sociais x Estresse:** Existe uma correlação entre o tempo gasto em redes sociais e o aumento do nível de estresse?
2.  **Redes Sociais x Felicidade:** O uso intensivo de plataformas digitais afeta o índice de felicidade reportado?
3.  **Redes Sociais x Sono:** O tempo de tela diário impacta a qualidade ou a duração do sono?
4.  **Hábitos Saudáveis:** Qual a relação entre a frequência de exercícios físicos, qualidade do sono e a felicidade geral?

## 3. Fonte de Dados
Os dados utilizados neste projeto são públicos e foram obtidos através do Kaggle.
* **Dataset:** Social Media and Mental Health Balance
* **Origem:** [Kaggle - Ayesha Imran](https://www.kaggle.com/datasets/ayeshaimran123/social-media-and-mental-health-balance/data)
* **Conteúdo:** O conjunto de dados inclui informações sobre tempo de tela, frequência de exercícios, métricas de sono, níveis de estresse e índices de felicidade de diversos usuários.

## 4. Metodologia e Arquitetura
O projeto segue a arquitetura de referência **Medallion (Bronze, Silver, Gold)**, implementada via **PySpark** e **Spark SQL**:

* **Camada Bronze (Raw):** Ingestão dos dados brutos (arquivo CSV) para o Data Lake sem transformações, garantindo a preservação do histórico original.
* **Camada Silver (Trusted):** Limpeza, padronização de tipos de dados, renomeação de colunas (tradução e formatação), tratamento de valores nulos e remoção de duplicatas.
* **Camada Gold (Refined):** Agregação dos dados em visões analíticas focadas nas perguntas de negócio, prontas para consumo por ferramentas de visualização.



## 1. Etapa
Para essa etapa vou conectar o arquivo importado e converter em um dataframe para a limpeza e organização. Esse é uma base de dados de nível 10n segundo o kaggle, logo é sabido que ela poderá estar limpa e sem dados faltantes. Por fim, para fins didáticos serão feitas todas as etapas de limpeza e organização.

In [0]:
# 1. Leitura da tabela bruta (Bronze)
table_name = "workspace.default.mental_health_and_social_media_balance_dataset"
df_bronze = spark.read.table(table_name)

# 2. Visualizar os dados e o Schema
display(df_bronze)
df_bronze.printSchema()

## 2. Etapa
Para essa etapa, identificaram-se os tipos de dados, se há dados nulos os "not a number", contagem dos dados, verificação da integridade e algumas estatísticas descritivas

In [0]:
from pyspark.sql.functions import col, count, when, isnan, desc
from pyspark.sql.types import DoubleType, FloatType

# --- A. Contagem de Nulos por coluna  ---
print("--- Contagem de Valores Nulos --- O objetivo é identificar se há valores nulos ou vazios em campos relevantes")

# Vamos criar uma lista de expressões dinamicamente verificando o tipo da coluna
expressoes_nulos = []

for campo in df_bronze.schema:
    nome_coluna = campo.name
    tipo_dado = campo.dataType
    
    # Se for número , checa nulo OU isNan(Not a number)
    if isinstance(tipo_dado, (DoubleType, FloatType)):
        expressoes_nulos.append(count(when(col(nome_coluna).isNull() | isnan(col(nome_coluna)), nome_coluna)).alias(nome_coluna))
    # Se for texto ou inteiro, checa apenas nulo (isNull) para não dar erro de Cast
    else:
        expressoes_nulos.append(count(when(col(nome_coluna).isNull() , nome_coluna)).alias(nome_coluna))

# Executa a contagem
df_bronze.select(expressoes_nulos).display()


# --- B. Verificar integridade dos campos categóricos (Ex: Gender) ---
print("--- Distribuição de Gênero --- Entender quais os tipos de genero, mas se os ")
# Verifica se a coluna Gender existe antes de tentar agrupar
if "Gender" in df_bronze.columns:
    df_bronze.groupBy("Gender").count().sort(desc("count")).display()


# --- C. Verificar integridade de campos numéricos ---
print("--- Estatísticas Descritivas ---")
df_bronze.describe().display()

### Resultado
Analisando os resultados, não há dados nulos ou não numéricos, ou seja, não precisarei eliminar dados ruins. Com base nos dados, a maioria dos dados são de homens, mas com uma boa representação percentual de mulheres, o que não gera um viés de "opiniões" masculinizadas. Com base nas estatísticas consegue-se entender a média da idade do público, que seria aproximadamente 33 anos, mas com uma variação de 16 a 49 anos. O tempo médio de telas é de 5,52 horas, ou seja, já é um tempo elevado e provavelmente grande parte das pessoas já utiliza demais as telas. Olhando superficialmente, o efeito da média de tempo de horas é refletido na baixa qualidade de sono do grupo. Porém, esse efeito não se repete no índice de felicidade. 



## 3. Etapa
Para essa etapa, identificaram-se os atributos da nova base de dados "Silver". Será feito um filtro simbólico para eliminar dados "nulos" de idade e dados com "tempo de tela" superior a um dia, que na prática seria impossível. 

In [0]:
from pyspark.sql.functions import col, trim, lower, initcap

# 1. Seleção e Renomeação (Mapeamento Bronze -> Silver)
df_silver = df_bronze.select(
    col("User_ID").alias("id_usuario"),
    col("Age").cast("integer").alias("idade"),
    
    # Padronizando Gênero: Remove espaços e coloca primeira letra maiúscula
    initcap(trim(col("Gender"))).alias("genero"),
    
    col("`Daily_Screen_Time(hrs)`").alias("tempo_tela_diario"),
    col("`Sleep_Quality(1-10)`").alias("qualidade_sono"),
    col("`Stress_Level(1-10)`").alias("nivel_estresse"),
    col("Days_Without_Social_Media").alias("dias_sem_redes"),
    col("`Exercise_Frequency(week)`").alias("freq_exercicio"),
    
    # Padronizando Plataforma
    initcap(trim(col("Social_Media_Platform"))).alias("plataforma_rede_social"),
    
    col("`Happiness_Index(1-10)`").alias("indice_felicidade")
)

# 2. Remoção de Duplicatas
df_silver = df_silver.dropDuplicates()

# 3. Filtros de Qualidade 
# Exemplo: Remover linhas onde a idade é nula ou tempo de tela é maior que 24h
df_silver = df_silver.filter(
    (col("idade").isNotNull()) & 
    (col("tempo_tela_diario") <= 24)
)

# Visualizar o resultado limpo
display(df_silver)

In [0]:
# Definir o caminho e nome da tabela Silver
# Sugiro manter no mesmo schema 'default' para facilitar
silver_table_name = "workspace.default.silver_social_media_health"

# Salvar como tabela Delta (formato padrão e otimizado do Databricks)
df_silver.write.format("delta").mode("overwrite").saveAsTable(silver_table_name)

print(f"Tabela Silver criada com sucesso: {silver_table_name}")

In [0]:
%sql
-- Ver se existe correlação visual rápida entre plataforma e felicidade
SELECT 
  plataforma_rede_social, 
  AVG(indice_felicidade) as media_felicidade,
  AVG(nivel_estresse) as media_estresse
FROM workspace.default.silver_social_media_health
GROUP BY plataforma_rede_social
ORDER BY media_felicidade DESC

1. Há correlação entre o uso de Redes Sociais e Estresse?
Esta query calcula a média de estresse por hora de tela e também o coeficiente de correlação estatística (que vai de -1 a 1).

In [0]:

%sql
-- Visão Agrupada (Para Gráfico de Linha ou Dispersão)
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(nivel_estresse), 2) as media_estresse,
  COUNT(*) as total_pessoas
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Cálculo Matemático da Correlação (-1 a 1)
-- Se próximo de 1: Quanto mais tela, mais estresse.
SELECT corr(tempo_tela_diario, nivel_estresse) as correlacao_tela_estresse
FROM workspace.default.silver_social_media_health;

2. Há correlação entre Felicidade e o uso de Redes Sociais?
Vamos ver se quem usa mais redes sociais é mais ou menos feliz.

In [0]:
%sql
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Correlação Estatística
SELECT corr(tempo_tela_diario, indice_felicidade) as correlacao_tela_felicidade
FROM workspace.default.silver_social_media_health;

3. Há correlação entre Redes Sociais e Sono?
Verificando se o tempo de tela afeta a qualidade do sono.

In [0]:
%sql
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(qualidade_sono), 2) as media_sono
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Correlação Estatística
SELECT corr(tempo_tela_diario, qualidade_sono) as correlacao_tela_sono
FROM workspace.default.silver_social_media_health;

4. Há correlação entre Sono, Felicidade e Exercícios?
Aqui cruzaremos a frequência de exercícios com o sono e a felicidade para ver o impacto de hábitos saudáveis.

In [0]:
%sql
-- Agrupando por frequência de exercício (Para Gráfico de Barras)
SELECT 
  freq_exercicio,
  ROUND(AVG(qualidade_sono), 2) as media_sono,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade,
  COUNT(*) as total_pessoas
FROM workspace.default.silver_social_media_health
GROUP BY freq_exercicio
ORDER BY freq_exercicio;

A camada Gold deve conter dados agregados e enriquecidos, prontos para serem consumidos por dashboards (Power BI, Tableau) ou para a apresentação final.

Vamos criar uma tabela Gold que resume o perfil dos usuários, criando "Faixas" (Buckets) para facilitar a análise, em vez de olhar usuário por usuário.

In [0]:
%sql
-- Apagar se já existir para refazer limpo
DROP TABLE IF EXISTS workspace.default.gold_analise_bem_estar;

-- Criar a tabela Gold com dados sumarizados por Perfil
CREATE TABLE workspace.default.gold_analise_bem_estar AS
SELECT
  genero,
  plataforma_rede_social,
  
  -- Criando uma categoria de uso para facilitar filtros no Dashboard
  CASE 
    WHEN tempo_tela_diario <= 2 THEN '1. Baixo Uso (0-2h)'
    WHEN tempo_tela_diario <= 5 THEN '2. Uso Moderado (2-5h)'
    WHEN tempo_tela_diario <= 8 THEN '3. Alto Uso (5-8h)'
    ELSE '4. Uso Extremo (>8h)'
  END AS categoria_uso_tela,
  
  -- Métricas Médias (KPIs)
  ROUND(AVG(nivel_estresse), 2) as media_estresse,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade,
  ROUND(AVG(qualidade_sono), 2) as media_sono,
  ROUND(AVG(freq_exercicio), 2) as media_exercicio,

  -- Contagem para saber a relevância estatística
  COUNT(*) as total_usuarios

FROM workspace.default.silver_social_media_health
GROUP BY 
  genero, 
  plataforma_rede_social, 
  categoria_uso_tela
ORDER BY 
  categoria_uso_tela, 
  plataforma_rede_social;

Após criar a Gold, faça um select simples para ver como ficou sua tabela final:

In [0]:
%sql
SELECT * FROM workspace.default.gold_analise_bem_estar

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Carregar os dados da tabela Gold para o Pandas
# O 'toPandas()' traz os dados para a memória do driver para plotagem
df_gold = spark.read.table("workspace.default.gold_analise_bem_estar").toPandas()

# Configurações visuais gerais
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 7) # Aumenta o tamanho padrão para melhor leitura

# --- GRÁFICO 1: Gênero e Média de Stress por Categoria de Uso de Tela ---
plt.figure()
sns.barplot(data=df_gold, x='categoria_uso_tela', y='media_estresse', hue='genero', palette='viridis')
plt.title('Média de Estresse por Categoria de Uso de Tela e Gênero', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Média de Estresse')
plt.xticks(rotation=45)
plt.legend(title='Gênero')
plt.tight_layout()
plt.show() # No Databricks, use display() se preferir ou plt.show()

# --- GRÁFICO 2: Gênero e Stress por Plataforma ---
plt.figure()
sns.barplot(data=df_gold, x='plataforma_rede_social', y='media_estresse', hue='genero', palette='magma')
plt.title('Média de Estresse por Plataforma de Rede Social e Gênero', fontsize=16)
plt.xlabel('Plataforma')
plt.ylabel('Média de Estresse')
plt.legend(title='Gênero')
plt.tight_layout()
plt.show()

# --- GRÁFICO 3: Sono e Categoria de Uso de Tela ---
plt.figure()
# Agrupamos para garantir que temos uma média única por categoria caso haja subdivisões
df_sono = df_gold.groupby('categoria_uso_tela')['media_sono'].mean().reset_index()
sns.barplot(data=df_sono, x='categoria_uso_tela', y='media_sono', palette='Blues_d')
plt.title('Média de Qualidade do Sono por Categoria de Uso de Tela', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Qualidade do Sono (Média)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- GRÁFICO 4: Categoria Uso Tela x Stress x Felicidade ---
plt.figure()
# Transformamos os dados para formato "longo" para o Seaborn plotar barras agrupadas das métricas
df_melted = df_gold.melt(id_vars=['categoria_uso_tela'], value_vars=['media_estresse', 'media_felicidade'], 
                         var_name='Métrica', value_name='Valor')
sns.barplot(data=df_melted, x='categoria_uso_tela', y='Valor', hue='Métrica', palette='coolwarm')
plt.title('Comparativo: Estresse vs Felicidade por Categoria de Uso', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Valor Médio (Escala 0-10)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- GRÁFICO 5: Stress, Sono e Gênero em relação à Felicidade ---
plt.figure()
sns.scatterplot(data=df_gold, x='media_estresse', y='media_felicidade', 
                hue='genero', size='media_sono', sizes=(50, 500), alpha=0.7, palette='deep')
plt.title('Relação Multivariada: Estresse vs Felicidade (Tamanho = Qualidade do Sono)', fontsize=16)
plt.xlabel('Média de Estresse')
plt.ylabel('Média de Felicidade')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Legenda')
plt.tight_layout()
plt.show()

# --- GRÁFICO 6: Rede Social com Uso de Tela e Felicidade ---
plt.figure()
sns.barplot(data=df_gold, x='plataforma_rede_social', y='media_felicidade', hue='categoria_uso_tela', palette='Set2')
plt.title('Felicidade por Plataforma e Intensidade de Uso', fontsize=16)
plt.xlabel('Plataforma')
plt.ylabel('Média de Felicidade')
plt.legend(title='Uso de Tela', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# --- GRÁFICO 7: Stress, Exercicio e Gênero em relação à Felicidade ---
plt.figure()
sns.scatterplot(data=df_gold, x='media_estresse', y='media_felicidade', 
                hue='genero', size='media_exercicio', sizes=(50, 500), alpha=0.7, palette='deep')
plt.title('Relação Multivariada: Estresse vs Felicidade (Tamanho = Exercicio)', fontsize=16)
plt.xlabel('Média de Estresse')
plt.ylabel('Média de Felicidade')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Legenda')
plt.tight_layout()
plt.show()